# Linearize the acrobot dynamics around the state x0
    so that obtain the linear approximation

In [31]:
import sympy
import scipy
import numpy as np
import matplotlib.pyplot as plt

from scipy.integrate import odeint
from collections import namedtuple
from sympy.abc import r, k, g, xi
from scipy.linalg import solve_continuous_are as solver


def Linearizing (r_const, g_const, xi_const, X_init):
    sympy.var('x0:4')
    X = sympy.Matrix(4,1,sympy.var('x0:4'))
    u = sympy.Matrix([sympy.var('u')])

    # inertia matrix
    M = sympy.Matrix ([[3 + 2*sympy.cos(x1), 1 + sympy.cos(x1)], [1 + sympy.cos(x1), 1]])
    
    # Coriolis, centripetal and gravitational forces
    c1 = x3*(2*x2+x3)*sympy.sin(x1)+2*g*sympy.sin(x0)+g*sympy.sin(x0+x1)
    c2 = -x2**2*sympy.sin(x1)+g*sympy.sin(x0+x1)
    
    # passive dynamics
    temp_a = sympy.zeros(4,4)
    temp_a[0:2, 0:2] = sympy.eye(2)
    temp_a[2:4, 2:4] = M.inv()
    a = temp_a * sympy.Matrix(4,1,[x2, x3, c1 - xi*x2, c2 - xi*x3])
    
    # control gain
    temp_b = sympy.zeros(4,4)
    temp_b[0:2, 0:2] = sympy.eye(2)
    temp_b[2:4, 2:4] = M.inv() 
    b = temp_b * sympy.Matrix(4,1, [0, 0, 0, 1])

    a = a.subs([(g, g_const), (xi, xi_const)])
    b = b.subs([(g, g_const), (xi, xi_const)])

    f = a + b*u #given the xi value and g value
    
    #linearization
    A = f.jacobian(X)
    B = f.jacobian(u)

    A0 = A.subs([(x0, X_init[0]),(x1, X_init[1]),(x2, X_init[2]),(x3, X_init[3])]) #given the equilibrium point
    B0 = B.subs([(x0, X_init[0]),(x1, X_init[1]),(x2, X_init[2]),(x3, X_init[3])]) #given the equilibrium point
    X_dot = A0 * X + B0 * u
    return a,b, A0, B0, X_dot


def Q_approximation (r_const, k_const):
    sympy.var('x0:4')
    X = sympy.Matrix(4,1,sympy.var('x0:4'))
    u = sympy.Matrix([sympy.var('u')])
    
    stage_cost =1- sympy.exp(k*sympy.cos(x0) + k*sympy.cos(x1) -2*k) + ((r**2)/2 * u*u)[0,0]
    mystage_cost = stage_cost.subs([(r,r_const), (k,k_const)])
    Q = sympy.hessian(mystage_cost, X)
    stage_cost0 = (0.5*X.T*Q*X + r**2*u**2/2) 
    return Q, stage_cost0
    
    
def LQR (Q, A0, B0, X_init):
    myQ = Q.subs([(x0,X_init[0]),(x1,X_init[1]),(x2,X_init[2]),(x3,X_init[3])])
    
    myQ_lqr = sympy.matrix2numpy(myQ).astype(float)
    A0_lqr = sympy.matrix2numpy(A0).astype(float)
    B0_lqr = sympy.matrix2numpy(B0).astype(float)
    
    M = solver(A0_lqr, B0_lqr, myQ_lqr, 1)
    return M

def nonlinear_system (y, t, a, b, my_pi):
    uu = my_pi(y[:, None])
    dydt = a.subs([(x0,y[0]),(x1,y[1]),(x2,y[2]),(x3,y[3])]) +\
    b.subs([(x0,y[0]),(x1,y[1]),(x2,y[2]),(x3,y[3])])*uu
    
    dydt = (sympy.matrix2numpy(dydt).astype(float)).squeeze()
    return dydt


sympy.var('x0:4')
X = sympy.Matrix(4,1,sympy.var('x0:4'))
u = sympy.Matrix([sympy.var('u')])
Consts = namedtuple('Consts', ['r_const', 'g_const', 'xi_const', 'k_const', 'X_init'])
myconsts = Consts(r_const = 1, g_const = 9.8, xi_const = 1, k_const = 1, X_init = np.array([0,0,0,0]))

a, b, A0, B0, X_dot = Linearizing (myconsts.r_const, myconsts.g_const, myconsts.xi_const, myconsts.X_init)
Q, stage_cost0 = Q_approximation (myconsts.r_const, myconsts.k_const)
M = LQR (Q, A0, B0, myconsts.X_init)

# construction on nonlinearized system
y0 = myconsts.X_init  #np.array([0,0,0,0]) #initial condition
t = np.linspace(0, 10, 101)
my_pi = lambda X: - 1/myconsts.r_const * B0.T * M * X
sol = odeint(nonlinear_system, y0, t, args=(a, b, my_pi))

M


array([[132230.69962896,  46306.40075231,  52361.25560798,
         20787.16780692],
       [ 46306.40075231,  16240.7415841 ,  18339.16402321,
          7283.20389965],
       [ 52361.25560798,  18339.16402321,  20734.52144416,
          8231.78258355],
       [ 20787.16780692,   7283.20389965,   8231.78258355,
          3268.37393004]])

# test 1

In [289]:
from scipy.integrate import odeint
import matplotlib.pyplot as plt

def nonlinear_system (y, t, a, b, my_pi):
    uu = my_pi(y[:, None])
    dydt = a.subs([(x0,y[0]),(x1,y[1]),(x2,y[2]),(x3,y[3])]) +\
    b.subs([(x0,y[0]),(x1,y[1]),(x2,y[2]),(x3,y[3])])*uu
    
    dydt = (sympy.matrix2numpy(dydt).astype(float)).squeeze()
    return dydt

y0 = np.array([0,0,0,0]) #initial condition
t = np.linspace(0, 10, 101)

my_pi = lambda X: - 1/r_const * B0.T * M * X
sol = odeint(nonlinear_system, y0, t, args=(a, b, my_pi))




plt.plot(t, sol[:, 0], 'b', label='theta(t)')
plt.plot(t, sol[:, 1], 'g', label='omega(t)')
plt.legend(loc='best')
plt.xlabel('t')
plt.grid()
plt.show()


# Test 2


In [246]:
import sympy
from scipy.linalg import solve_continuous_are as solver
import numpy as np



from sympy.abc import r, k, g, xi

sympy.var('x0:4')
#sympy.var('u')
X = sympy.Matrix(4,1,sympy.var('x0:4'))
u = sympy.Matrix([sympy.var('u')])

# inertia matrix
M = sympy.Matrix ([[3 + 2*sympy.cos(x1), 1 + sympy.cos(x1)], [1 + sympy.cos(x1), 1]])
# Coriolis, centripetal and gravitational forces
c1 = x3*(2*x2+x3)*sympy.sin(x1)+2*g*sympy.sin(x0)+g*sympy.sin(x0+x1)
c2 = -x2**2*sympy.sin(x1)+g*sympy.sin(x0+x1)
# passive dynamics
temp_a = sympy.zeros(4,4)
temp_a[0:2, 0:2] = sympy.eye(2)
temp_a[2:4, 2:4] = M.inv()
a = temp_a * sympy.Matrix(4,1,[x2, x3, c1 - xi*x2, c2 - xi*x3])
# control gain
temp_b = sympy.zeros(4,4)
temp_b[0:2, 0:2] = sympy.eye(2)
temp_b[2:4, 2:4] = M.inv() 
b = temp_b * sympy.Matrix(4,1, [0, 0, 0, 1])

f = (a + b*u).subs([(g,9.8), (xi, 1)])
#linearization
A = f.jacobian(X)
B = f.jacobian(u)

A0 = A.subs([(x0,0),(x1,0),(x2,0),(x3,0)]) #given the equilibrium point
B0 = B.subs([(x0,0),(x1,0),(x2,0),(x3,0)]) #given the equilibrium point

X_dot = A0 * X + B0 * u




# Compute a quadratic approximation to the stage cost in the form:

In [233]:
g =1- sympy.exp(k*sympy.cos(x0) + k*sympy.cos(x1) -2*k) + ((r**2)/2 * u*u)[0,0]
myg = g.subs([(r,1), (k,1)])

Q = sympy.hessian(myg, X)

my_g0 = (0.5*X.T*Q*X + r**2*u**2/2)
my_g0

Matrix([[r**2*u**2/2 + x0*(0.5*x0*(-exp(cos(x0) + cos(x1) - 2)*sin(x0)**2 + exp(cos(x0) + cos(x1) - 2)*cos(x0)) - 0.5*x1*exp(cos(x0) + cos(x1) - 2)*sin(x0)*sin(x1)) + x1*(-0.5*x0*exp(cos(x0) + cos(x1) - 2)*sin(x0)*sin(x1) + 0.5*x1*(-exp(cos(x0) + cos(x1) - 2)*sin(x1)**2 + exp(cos(x0) + cos(x1) - 2)*cos(x1)))]])

In [ ]:

def Q_approximation (r_const, k_const):
    stage_cost =1- sympy.exp(k*sympy.cos(x0) + k*sympy.cos(x1) -2*k) + ((r**2)/2 * u*u)[0,0]
    mystage_cost = stage_cost.subs([(r,r_const), (k,k_const)])
    Q = sympy.hessian(mystage_cost, X)
    
    

In [237]:
from scipy.linalg import solve_continuous_are as solver
from sympy import lambdify
import numpy as np

myQ = Q.subs([(x0,0),(x1,0),(x2,0),(x3,0)])

A0_lqr = sympy.matrix2numpy(A0).astype(float)
B0_lqr = sympy.matrix2numpy(B0).astype(float)
myQ_lqr = sympy.matrix2numpy(myQ).astype(float)
M = solver(A0_lqr, B0_lqr, myQ_lqr, 1)


my_pi = - 1/r * B0.T * M * X



my_pi





Matrix([[786.672181344911*x0/r + 262.308548172979*x1/r + 310.129970596216*x2/r + 121.695516898224*x3/r]])

In [28]:
x_init = np.random.randn(4)
x_init.reshape((4,1))
x_init

array([-0.9954949 , -0.52216536, -0.42512278,  0.05655485])